<a href="https://colab.research.google.com/github/marcelo7bastos/dados_aberto_pgc_pca/blob/main/pgc_pca_itens_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análise exploratória de dados (EDA) com o objetivo de criar um ranking dos itens de PGC/PCA

1. Definição do Problema e Planejamento Objetivo;
2. Metodologia e Ferramentas
3. Coleta de Dados
a. Consulta de CNPJs:
b. Consulta de PGC e Itens por Classe:
3. Limpeza e Preparação dos Dados
Limpeza:
Estruturação:
4. Análise Exploratória de Dados (EDA)
Análise Univariada:
Análise Bivariada/Multivariada:
Visualização:
5. Criação do Ranking
5.1. Agrupamento:
5.2. Ordenação:
Seleção: Selecione os top 20 (ou mais) itens por classe, com base no critério definido (e.g., volume planejado, frequência de planejamento).
6. Avaliação e Interpretação
7. Validação
8. Compartilhamento dos Resultados - Relatório

Perspectivas:

* Governos
  * planejamento
    * políticas públicas, exemplo
  * execução
  * controle


* Sociedade

* Empresa

* testes iniciais
  * Pareto = 20 itens!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: nbextensions
#testar
!pip install jupyter_nbextensions_configurator
!jupyter nbextensions_configurator enable --user


### 1 - Definição do Problema e Planejamento Objetivo:
Definir claramente o que se quer alcançar. No seu caso, é criar um ranking dos itens mais planejados por classe em 2023. Fonte de Dados: Identificar as fontes de dados necessárias. Você já identificou as APIs relevantes.

# Metodologia e Ferramentas

## Metodologia


Para realizar uma análise exploratória de dados (EDA) com o objetivo de criar um ranking dos itens (por classe) mais planejados em 2023 usando as APIs de dados abertos do governo brasileiro para consultas de compras governamentais, siga os passos detalhados abaixo seguidos de maneira iterada e incremental:

Pseudocódigo

  Definir função consultar_pgc_detalhe com retorno json pgc_detalhe por página

  Definir função consultar_orgão com retorno json orgão por página

  executar Estrutura de repetição que percorrer todas as páginas LISTAR OS ÓRGÃOS
  resultado lista_cnpjCpfOrgao

  executar Estrutura de repetição para consultar e guardar o pgc_detalhe por órgão
  resultado dataframe e arquivo csv

  executar criação de relatórios contendo:
  indicador de quantos itens, quantos órgãos com informação e sem informação
  
  Criar 2 rankings considerando o tipoItem (S ou M)
  Comparar itemSustentavel com o todo
  Criar 2 rankings considerando o tipoItem (S ou M) com filtro itemSustentavel

##Ferramentas

In [ ]:
#!pip install tqdm

# Import
import requests
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm  # Importa o tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Definição das Funções

#### Funções para gerar lista de CNPJ de ógãos

1. Ler os dados em: "GET /modulo-uasg/2_consultarOrgao

https://dadosabertos.compras.gov.br/swagger-ui/index.html#/5%20-%20UASG/consultarOrgao


acrescentar: https://dadosabertos.compras.gov.br/modulo-uasg/2_consultarOrgao?pagina=1&statusOrgao=true&usoSisg=true

In [ ]:
def get_2_consultar_orgao(pagina, tamanho_pagina):
    """
    Faz uma solicitação GET para consultar órgãos da API de Dados Abertos do Governo.

    Args:
        pagina (int): Número da página de resultados a ser consultada.
        tamanho_pagina (int): Número de registros por página.

    Returns:
        response (requests.Response): Objeto de resposta contendo os dados JSON, se a solicitação for bem-sucedida.
    """
    url = "https://dadosabertos.compras.gov.br/modulo-uasg/2_consultarOrgao"
    params = {
        "pagina": pagina,
        "tamanhoPagina": tamanho_pagina,
        "statusOrgao": "true", #Seria necessário consultar statusOrgao False?
        "usoSisg": "true"
    }
    headers = {"accept": "*/*"}
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Levanta uma exceção para respostas de erro (4xx, 5xx)
        return response
    except requests.exceptions.HTTPError as e:
        print(f"Erro HTTP: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a solicitação: {e}")

In [ ]:
get_2_consultar_orgao(1,50)

<Response [200]>

In [ ]:
def listar_orgaos(tamanho_pagina=500):
    """
    Lista todos os órgãos consultando todas as páginas disponíveis.

    Args:
        tamanho_pagina: Quantidade de registros por página.

    Returns:
        DataFrame com os dados de todos os órgãos listados.
    """
    pagina = 1
    df_resultado_orgao = pd.DataFrame(columns=['nomeOrgao', 'cnpjCpfOrgao'])
    while True:
        data = get_2_consultar_orgao(pagina, tamanho_pagina).json()
        if data and "resultado" in data:
            df_temp = pd.DataFrame(data["resultado"], columns=['nomeOrgao', 'cnpjCpfOrgao'])
            df_resultado_orgao = pd.concat([df_resultado_orgao, df_temp], ignore_index=True)
            print(f"Página {pagina} processada.")
            pagina += 1
            if pagina > data.get("totalPaginas", 0):
                break
        else:
            print("Erro ao processar dados ou chave 'resultado' não encontrada.")
            break
    return df_resultado_orgao

In [ ]:
def lista_cnpjCpfOrgao_validos():
    """
    Filtra os CNPJs válidos (diferentes de 0) do DataFrame retornado pela função listar_orgaos
    e retorna uma lista dos CNPJs/CPFs válidos, removendo duplicatas.

    Returns:
        list: Lista contendo os CNPJs/CPFs válidos dos órgãos, sem duplicatas.
    """
    df = listar_orgaos()  # Obtém o DataFrame com os dados dos órgãos
    coluna = 'cnpjCpfOrgao'  # Define a coluna a ser filtrada

    # Filtra o DataFrame para incluir apenas linhas onde o CNPJ/CPF é diferente de '0'
    df_filtrado = df[df[coluna] != '0']

    # Remove duplicatas baseado na coluna de CNPJs/CPFs
    df_filtrado_sem_duplicatas = df_filtrado.drop_duplicates(subset=[coluna])

    # Converte a coluna de CNPJs/CPFs filtrada e sem duplicatas em uma lista
    lista_cnpj_validos = df_filtrado_sem_duplicatas[coluna].tolist()

    return lista_cnpj_validos

In [ ]:
# Executando a listagem de órgãos e transformando em lista de CNPJs
#df_orgaos = listar_orgaos()

In [ ]:
### Utilizando a função para obter a lista de CNPJs/CPFs válidos
#lista_cnpj_validos = lista_cnpjCpfOrgao_validos()
#lista_cnpj_validos
# Exibindo a lista filtrada
#print(lista_cnpj_validos)
#len(lista_cnpj_validos)

#lista_cnpj_validos.count('00394452000103')
#df_orgaos

#### Funções para ler os dados em: "GET /modulo-pgc/1_consultarPgcDetalhe



origem dos dados: https://dadosabertos.compras.gov.br/swagger-ui/index.html#/4%20-%20PGC/consultarPgcDetalhe


In [ ]:
def get_1_consultar_pgc_detalhe(orgao, ano_pca_projeto_compra, pagina, tamanho_pagina):
    """
    Faz uma solicitação GET à API de Dados Abertos do Governo para consultar detalhes do PGC (Plano de Contratações Anuais) de um determinado órgão.

    Args:
        orgao (str): Identificador do órgão para o qual os detalhes do PGC serão consultados.
        ano_pca_projeto_compra (int): Ano do Plano de Contratações Anuais a ser consultado.
        pagina (int): Número da página de resultados a ser consultada.
        tamanho_pagina (int): Número de registros por página.

    Returns:
        requests.Response: Objeto de resposta contendo os dados JSON da página solicitada, se a solicitação for bem-sucedida.

    Raises:
        requests.exceptions.HTTPError: Se a solicitação HTTP resultar em um erro de cliente (4xx) ou servidor (5xx).
        requests.exceptions.RequestException: Para outros tipos de erros de solicitação, como problemas de conexão.

    Exemplo:
        resposta = get_1_consultar_pgc_detalhe('00394452000103', 2023, 1, 50)
        dados = resposta.json()
        print(dados)
    """
    url = "https://dadosabertos.compras.gov.br/modulo-pgc/1_consultarPgcDetalhe"
    params = {
        "orgao": orgao,
        "anoPcaProjetoCompra": ano_pca_projeto_compra,
        "pagina": pagina,
        "tamanhoPagina": tamanho_pagina
    }
    headers = {"accept": "*/*"}

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Levanta uma exceção para respostas de erro
        return response
    except requests.exceptions.HTTPError as e:
        print(f"Erro HTTP: {e}")
        raise
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a solicitação: {e}")
        raise


In [ ]:
resposta = get_1_consultar_pgc_detalhe('00394494000136', 2023, 1, 50)
dados = resposta.json()
print(dados)

{'resultado': [{'codigoUasg': '200601', 'nomeUasg': 'PENITENCIÁRIA FEDERAL EM CATANDUVAS - PR', 'orgao': '00394494000136', 'numeroArtefato': 2, 'anoArtefato': 2023, 'codigoEstadoArtefato': 9, 'codigoCategoriaArtefato': 1, 'descricaoArtefato': '2/2023', 'codigoTipoArtefato': 5, 'ordemDfd': 1, 'descricaoObjetoDfd': 'Aquisição de materiais pedagógicos para atender a necessidade da Divisão de Reabilitação', 'nivelPrioridadeDfd': 1, 'dataPrevistaFormalizacaoDemanda': '2023-09-01T00:00:00', 'codigoAreaDfd': '11885', 'tipoItem': 'M', 'itemSustentavel': None, 'codigoGrupoMaterial': 75, 'nomeGrupoMaterial': 'UTENSÍLIOS DE ESCRITÓRIO E MATERIAL DE EXPEDIENTE', 'codigoClasseMaterial': 7510, 'nomeClasseMaterial': 'ARTIGOS PARA ESCRITÓRIO', 'codigoPdmMaterial': 196, 'nomePdmMaterial': 'CADERNO', 'codigoSecaoServico': None, 'nomeSecaoServico': None, 'codigoDivisaoServico': None, 'nomeDivisaoServico': None, 'codigoGrupoServico': None, 'nomeGrupoServico': None, 'codigoClasseServico': None, 'nomeClasse

In [ ]:
display(dados['totalRegistros'])
display(dados['totalPaginas'])

8401

169

### Extrair dados

#### Teste Extrair CNPJ e total de Registros

df_total_registros

In [ ]:
lista_cnpjCpfOrgao_validos()


Erro HTTP: 500 Server Error:  for url: https://dadosabertos.compras.gov.br/modulo-uasg/2_consultarOrgao?pagina=1&tamanhoPagina=500&statusOrgao=true&usoSisg=true


AttributeError: 'NoneType' object has no attribute 'json'

In [ ]:
#teste de código - 20240228

#import pandas as pd
#import requests
#from tqdm import tqdm
from time import sleep

# Supondo que estas são as funções e variáveis definidas em algum lugar do seu código
orgaos = lista_cnpjCpfOrgao_validos()
anos_pca_projeto_compra = [2023]
tamanho_pagina = 50

# Lista para armazenar os dados coletados
data_total_registros = []
# Lista para armazenar os erros
erros = []

# Função para realizar a chamada à API com retentativas
def chamar_api_com_retentativa(orgao, ano, pagina, tamanho_pagina, max_tentativas=6):
    tentativas = 0
    while tentativas < max_tentativas:
        try:
            response = get_1_consultar_pgc_detalhe(orgao, ano, pagina, tamanho_pagina)
            if response.status_code == 200:
                return response
            else:
                print(f"Tentativa {tentativas + 1} falhou: Código de status {response.status_code}")
                tentativas += 1
                sleep(1)  # Espera 1 segundo antes da próxima tentativa
        except requests.exceptions.RequestException as e:
            print(f"Tentativa {tentativas + 1} falhou: {e}")
            tentativas += 1
            sleep(1)
    return None

# Loop pelos anos com barra de progresso do tqdm
for ano in tqdm(anos_pca_projeto_compra, desc='Anos'):
    # Loop pelos órgãos com barra de progresso do tqdm
    for orgao in tqdm(orgaos, desc='Órgãos', leave=False):
        print(f"Processando órgão {orgao} para o ano {ano}")
        pagina = 1  # Definindo a página inicial

        # Chama a API com retentativas
        response = chamar_api_com_retentativa(orgao, ano, pagina, tamanho_pagina)

        if response and response.status_code == 200:
            data = response.json()
            total_registros = data.get("totalRegistros", 0)  # Usa .get para evitar KeyError

            # Adiciona o total de registros coletado à lista
            data_total_registros.append({
                "ano": ano,
                "orgao": orgao,
                "totalRegistros": total_registros
            })
        else:
            erro = {"ano": ano, "orgao": orgao, "erro": "Falha após máximas tentativas ou erro de rede"}
            erros.append(erro)
            print(f"Erro registrado para o órgão {orgao} no ano {ano}")

# Criação dos DataFrames
df_total_registros = pd.DataFrame(data_total_registros)
df_erros = pd.DataFrame(erros)

# Exibição do DataFrame dos registros
print(df_total_registros)
# Exibição do DataFrame dos erros
print(df_erros)


Página 1 processada.
Página 2 processada.
Página 3 processada.
Página 4 processada.


Órgãos:   0%|          | 0/1660 [00:00<?, ?it/s]

Processando órgão 9773169000159 para o ano 2023



Órgãos:   0%|          | 1/1660 [00:01<28:44,  1.04s/it]

Processando órgão 974863000160 para o ano 2023


...

Processando órgão 00015792000178 para o ano 2023



Órgãos: 100%|█████████▉| 1659/1660 [33:08<00:01,  1.08s/it]

Processando órgão 00000368000150 para o ano 2023



Anos: 100%|██████████| 1/1 [33:09<00:00, 1989.89s/it]

       ano           orgao  totalRegistros
0     2023   9773169000159               0
1     2023    974863000160               0
2     2023  97229181000164               0
3     2023  96480850000103               0
4     2023  96291141000180               0
...    ...             ...             ...
1655  2023  00105650000100               0
1656  2023  00094821000134               0
1657  2023  00084749000164               0
1658  2023  00015792000178               0
1659  2023  00000368000150               0

[1660 rows x 3 columns]
Empty DataFrame
Columns: []
Index: []


In [ ]:
display(df_total_registros)

,ano,orgao,totalRegistros
0,2023,9773169000159,0
1,2023,974863000160,0
2,2023,97229181000164,0
3,2023,96480850000103,0
4,2023,96291141000180,0
...,...,...,...
1655,2023,00105650000100,0
1656,2023,00094821000134,0
1657,2023,00084749000164,0
1658,2023,00015792000178,0


In [ ]:
# prompt: expostar df_total_registros para csv

df_total_registros.to_csv('df_total_registros.csv')


In [ ]:
# prompt: transformar df_total_registros[orgao] em uma lista considerando apenas os orgaos que tem total de registros maior que 0

lista_orgaos_c_pgc = df_total_registros[df_total_registros['totalRegistros'] > 0]['orgao'].tolist()

In [ ]:
# prompt: contar uma lista
print(len(df_total_registros))
print(len(lista_orgaos_c_pgc))


1660
173


In [ ]:
# Definição dos dados da pesquisa
#orgaos = ['00394494000136', '9773169000159']  # Apenas para teste
#orgaos = lista_cnpjCpfOrgao_validos() #todos os cnpjs não nulos
orgaos = lista_orgaos_c_pgc #lista de órgãos que efetivamente tem pgc
#anos_pca_projeto_compra = [2022, 2023, 2024]
anos_pca_projeto_compra = [2023]
tamanho_pagina = 50

# Lista para armazenar os dados coletados
data_total_registros = []

# Loop pelos anos com barra de progresso do tqdm
for ano in tqdm(anos_pca_projeto_compra, desc='Anos'):
    # Loop pelos órgãos com barra de progresso do tqdm
    for orgao in tqdm(orgaos, desc='orgaos', leave=False):
        print(f"Processando órgão {orgao} para o ano {ano}")
        pagina = 1  # Definindo a página inicial

        # Chama a API para a primeira página apenas para obter o total de registros
        response = get_1_consultar_pgc_detalhe(orgao, ano, pagina, tamanho_pagina)
        if response.status_code == 200:
            data = response.json()
            total_registros = data["totalRegistros"]

            # Adiciona o total de registros coletado à lista
            data_total_registros.append({
                "ano": ano,
                "orgao": orgao,
                "totalRegistros": total_registros
            })
        else:
            print(f"Erro na solicitação ao órgão {orgao} para o ano {ano}: {response.status_code}")

# Criação do DataFrame
df_total_registros = pd.DataFrame(data_total_registros)

# Exibição do DataFrame
print(df_total_registros)


orgaos:   0%|          | 0/173 [00:00<?, ?it/s]

Processando órgão 95591764000105 para o ano 2023



orgaos:   1%|          | 1/173 [00:01<05:11,  1.81s/it]

Processando órgão 94877586000110 para o ano 2023



orgaos:   1%|          | 2/173 [00:03<05:13,  1.84s/it]

...

Processando órgão 00394411000109 para o ano 2023



orgaos:  99%|█████████▉| 172/173 [05:43<00:02,  2.49s/it]

Processando órgão 00119784000171 para o ano 2023



Anos: 100%|██████████| 1/1 [05:45<00:00, 345.53s/it]

      ano           orgao  totalRegistros
0    2023  95591764000105             460
1    2023  94877586000110             877
2    2023  92969856000198            1058
3    2023  92967595000177              50
4    2023  92242080000100            2406
..    ...             ...             ...
168  2023  00394460000141            5611
169  2023  00394452000103          126843
170  2023  00394445000101             892
171  2023  00394411000109            1293
172  2023  00119784000171              62

[173 rows x 3 columns]


In [ ]:
# Exibição do DataFrame
print(orgaos)

#### Extrair Itens:


'codigoClasseMaterial','nomeClasseMaterial','codigoGrupoServico','nomeGrupoServico'

Exemplos: Processando órgão 00394445000101 para o ano 2023
Página 1 de 2 processada.
Página 2 de 2 processada.
Processando órgão 00394411000109 para o ano 2023
Página 1 de 3 processada.
Página 2 de 3 processada.
Página 3 de 3 processada.
Processando órgão 00119784000171 para o ano 2023
Página 1 de 1 processada.


In [ ]:
# Definição dos dados da pesquisa - executado em 4207s (1h10)
anos_pca_projeto_compra = [2023]
#orgaos = lista_cnpjCpfOrgao_validos()
orgaos = lista_orgaos_c_pgc #lista de órgãos que efetivamente tem pgc
#orgaos = ['00394494000136', '9773169000159'] #apenas teste
tamanho_pagina = 500
colunas_interesse = ['tipoItem','codigoClasseMaterial','nomeClasseMaterial','codigoGrupoServico','nomeGrupoServico', 'itemSustentavel', 'codigoItemCatalogo', 'descricaoItemCatalogo', 'valorTotalItem', 'anoPcaProjetoCompra']

# Inicialização do arquivo CSV
if os.path.exists('items_planejados.csv'):
    os.remove('items_planejados.csv')  # Remove o arquivo se já existir para começar do zero

for ano in anos_pca_projeto_compra:
    for orgao in orgaos:
        print(f"Processando órgão {orgao} para o ano {ano}")
        pagina = 1

        while True:
            try:
                response = get_1_consultar_pgc_detalhe(orgao, ano, pagina, tamanho_pagina)
                if response.status_code == 200:
                    data = response.json()
                    if "resultado" in data:
                        df_temp = pd.DataFrame(data["resultado"], columns=colunas_interesse[:-1])  # Colunas, excluindo ano
                        df_temp['anoPcaProjetoCompra'] = ano  # Adiciona coluna de ano

                        # Teste se 'codigoItemCatalogo' é nulo e descarte essas linhas
                        df_temp = df_temp.dropna(subset=['codigoItemCatalogo'])

                        if not df_temp.empty:
                            df_temp.to_csv('items_planejados.csv', mode='a', header=not os.path.exists('items_planejados.csv'), index=False)

                        total_paginas = data["totalPaginas"]
                        print(f"Página {pagina} de {total_paginas} processada.")
                    else:
                        print("Erro: chave 'resultado' não encontrada.")
                        break
                else:
                    print(f"Erro na solicitação: {response.status_code}")
                    break
            except Exception as e:
                print(f"Erro ao processar os dados: {e}")
                break

            pagina += 1
            if pagina > total_paginas:
                break

Processando órgão 95591764000105 para o ano 2023
Página 1 de 1 processada.
Processando órgão 94877586000110 para o ano 2023
Página 1 de 2 processada.
Página 2 de 2 processada.
Processando órgão 92969856000198 para o ano 2023
Página 1 de 3 processada.
Página 2 de 3 processada.
Página 3 de 3 processada.
Processando órgão 92967595000177 para o ano 2023
Página 1 de 1 processada.
Processando órgão 92242080000100 para o ano 2023
Página 1 de 5 processada.
Página 2 de 5 processada.
Página 3 de 5 processada.
Página 4 de 5 processada.
Página 5 de 5 processada.
Processando órgão 86900545000170 para o ano 2023
Página 1 de 1 processada.
Processando órgão 83899526000182 para o ano 2023
Página 1 de 5 processada.
Página 2 de 5 processada.
Página 3 de 5 processada.
Página 4 de 5 processada.
Página 5 de 5 processada.
Processando órgão 76639384000159 para o ano 2023
Página 1 de 1 processada.
Processando órgão 75101873000190 para o ano 2023
Página 1 de 2 processada.
Página 2 de 2 processada.
Processando ó

# Análise dos Resultados

## Carrega dados

In [ ]:
# carregar planilha armazenada no local: /content/drive/MyDrive/MGI (1)/pgc_pca/items_planejados.csv
df_itens_planejados = pd.read_csv("/content/drive/MyDrive/MGI (1)/pgc_pca/items_planejados_20240302.csv")


In [ ]:
display(df_itens_planejados)

,tipoItem,codigoClasseMaterial,nomeClasseMaterial,codigoGrupoServico,nomeGrupoServico,itemSustentavel,codigoItemCatalogo,descricaoItemCatalogo,valorTotalItem,anoPcaProjetoCompra
0,S,NaN,NaN,181.0,"SERVICOS DE ARRENDAMENTO MERCANTIL, FINANCEIRO...",NaN,27618,ALUGUEL DE IMPRESSORA / MULTIFUNCIONAL / PLOTT...,72600.00,2023
1,S,NaN,NaN,871.0,SERVIÇOS DE MANUTENÇÃO E REPARO DE PRODUTOS FA...,NaN,2658,MANUTENCAO / INSTALACAO - NOBREAK,59040.00,2023
2,S,NaN,NaN,871.0,SERVIÇOS DE MANUTENÇÃO E REPARO DE PRODUTOS FA...,NaN,22454,AR CONDICIONADO - MANUTENCAO SISTEMA CENTRAL,56760.00,2023
3,S,NaN,NaN,182.0,SERVIÇOS DE LICENCIAMENTO E CONTRATOS DE TRA...,NaN,27502,CESSAO TEMPORARIA DE DIREITOS SOBRE PROGRAMAS ...,100320.00,2023
4,M,6310.0,SISTEMA DE SINALIZAÇÃO DE TRÁFEGO E TRÂNSITO,NaN,NaN,False,258653,"CONE SINALIZAÇÃO, MATERIAL: POLIPROPILENO , AL...",3975.00,2023
...,...,...,...,...,...,...,...,...,...,...
357226,M,7210.0,UTENSÍLIOS DOMÉSTICOS,NaN,NaN,False,459468,"COLCHÃO, TIPO: ESPUMA , MATERIAL: ESPUMA POLIU...",39553.50,2023
357227,M,7105.0,MOBILIÁRIO DOMÉSTICO,NaN,NaN,False,481041,"CAMA CONJUGADA, MATERIAL: METAL , COMPRIMENTO:...",82306.29,2023
357228,S,NaN,NaN,166.0,SERVIÇOS DE MANUTENÇÃO E INSTALAÇÃO DE EQUIPAM...,NaN,27103,SERVICOS DE MANUTENCAO E REPARACAO DE COMPUTAD...,57000.00,2023
357229,S,NaN,NaN,643.0,SERVIÇOS DE TRANSPORTE RODOVIÁRIO,NaN,3263,TRANSPORTE RODOVIARIO - CARGAS / ENCOMENDAS,57000.00,2023


## MATERIAIS COM MAIORES VALORES PARA AQUISIÇÃO

In [ ]:
# Agrupar por 'codigoClasseMaterial', somar os 'valorTotalItem', e pegar a primeira 'descricaoItemCatalogo'
df_grupo_itens = df_itens_planejados.groupby('codigoClasseMaterial').agg({
    'valorTotalItem': 'sum',
    'nomeClasseMaterial': 'first'  # Assume que a descrição é constante para cada código de item classe material
}).reset_index()

# Renomear a coluna 'valorTotalItem' para 'somaValorTotalItem'
df_grupo_itens.rename(columns={'valorTotalItem': 'somaValorTotalItem'}, inplace=True)


# Ordenar o resultado do agrupamento pelo 'somaValorTotalItem' em ordem decrescente e selecionar os top 20
top_20_itens_agrupados = df_grupo_itens.sort_values(by='somaValorTotalItem', ascending=False).head(20)

#corrigindo formatação!
top_20_itens_agrupados['somaValorTotalItem'] = top_20_itens_agrupados['somaValorTotalItem'].apply(lambda x: f"{x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

display(top_20_itens_agrupados)

,codigoClasseMaterial,somaValorTotalItem,nomeClasseMaterial
268,6505.0,"21.028.426.295,73",DROGAS E MEDICAMENTOS
279,6550.0,"1.723.608.286,16","SUBSTÂNCIAS PARA DIAGNÓSTICO ""IN VITRO"", REAG..."
19,2320.0,"1.708.477.408,56",VEÍCULOS SOBRE RODAS
404,8710.0,"1.404.450.900,03",FORRAGENS E ALIMENTOS
315,7010.0,"836.925.284,45",COMPUTADORES
309,6840.0,"790.319.018,38",PESTICIDAS E DESINFETANTES
272,6515.0,"687.381.957,54","INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS MÉDIC..."
409,8905.0,"407.886.732,82","CARNES, AVES E PEIXES"
199,5680.0,"357.756.797,27",MATERIAIS DIVERSOS PARA CONSTRUÇÃO
106,3805.0,"297.593.167,78",EQUIPAMENTOS PARA MOVIMENTAÇÃO E ESCAVAÇÃO DE ...


##SERVIÇOS MARIORES VALORES PLANEJADOS PARA AQUISIÇÃO




In [ ]:
# Agrupar por 'codigoGrupoServico', somar os 'valorTotalItem', e pegar a primeira 'descricaoItemCatalogo'
df_grupo_itens = df_itens_planejados.groupby('codigoGrupoServico').agg({
    'valorTotalItem': 'sum',
    'nomeGrupoServico': 'first'  # Assume que a descrição é constante para cada código de item classe material
}).reset_index()

# Renomear a coluna 'valorTotalItem' para 'somaValorTotalItem'
df_grupo_itens.rename(columns={'valorTotalItem': 'somaValorTotalItem'}, inplace=True)


# Ordenar o resultado do agrupamento pelo 'somaValorTotalItem' em ordem decrescente e selecionar os top 20
top_20_itens_agrupados = df_grupo_itens.sort_values(by='somaValorTotalItem', ascending=False).head(20)

#corrigindo formatação!
top_20_itens_agrupados['somaValorTotalItem'] = top_20_itens_agrupados['somaValorTotalItem'].apply(lambda x: f"{x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))

display(top_20_itens_agrupados)

,codigoGrupoServico,somaValorTotalItem,nomeGrupoServico
92,911.0,"15.764.852.455,79",SERVIÇOS ADMINISTRATIVOS DO GOVERNO
28,542.0,"13.827.949.067,30",SERVIÇOS GERAIS DE CONSTRUÇÃO PARA OBRAS DE EN...
31,545.0,"5.182.651.889,69",TIPOS ESPECIAIS DE SERVIÇOS DE CONSTRUÇÃO
27,541.0,"5.048.276.952,25",SERVIÇOS GERAIS DE CONSTRUÇÃO DOS EDIFÍCIOS
61,732.0,"3.059.700.292,04",SERVIÇOS DE LEASING OU ALUGUEL RELACIONADOS A ...
66,833.0,"2.578.742.345,04",SERVIÇOS DE ENGENHARIA
81,859.0,"1.948.085.298,43",OUTROS SERVIÇOS DE SUPORTE
17,165.0,"1.002.017.416,08",SERVICOS PARA A INFRAESTRUTURA DE TECNOLOGIA...
1,112.0,"975.520.024,95",SERVIÇOS DE MANUTENÇÃO E SUSTENTAÇÃO DE SOFTWARE
46,672.0,"849.659.593,57",SERVIÇOS DE ARMAZENAGEM


from matplotlib import pyplot as plt
top_20_itens_agrupados['codigoGrupoServico'].plot(kind='hist', bins=20, title='codigoGrupoServico')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
top_20_itens_agrupados['somaValorTotalItem'].plot(kind='hist', bins=20, title='somaValorTotalItem')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
top_20_itens_agrupados.plot(kind='scatter', x='codigoGrupoServico', y='somaValorTotalItem', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
top_20_itens_agrupados['codigoGrupoServico'].plot(kind='line', figsize=(8, 4), title='codigoGrupoServico')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
top_20_itens_agrupados['somaValorTotalItem'].plot(kind='line', figsize=(8, 4), title='somaValorTotalItem')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# @title codigoItemCatalogo

from matplotlib import pyplot as plt
top_20_itens_agrupados['codigoItemCatalogo'].plot(kind='hist', bins=20, title='codigoItemCatalogo')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Converter somaValorTotalItem de volta para float para plotagem
top_20_itens_agrupados['somaValorTotalItem'] = top_20_itens_agrupados['somaValorTotalItem'].astype(float)

# Criar um gráfico de barras
plt.figure(figsize=(10, 8))
plt.barh(top_20_itens_agrupados['descricaoItemCatalogo'], top_20_itens_agrupados['somaValorTotalItem'])
plt.xlabel('Valor Total')
plt.ylabel('Descrição do Item')
plt.title('Top 20 Itens por Valor Total')
plt.gca().invert_yaxis()  # Inverter o eixo y para mostrar o maior valor no topo
plt.show()


ValueError: could not convert string to float: '15.764.852.455,79'

próximos passos

0 Otimizar consultas

indicador de quantos itens, quantos órgãos com informação e sem informação

1 Criar 2 rankings considerando o tipoItem (S ou M)

2 Comparar itemSustentavel com o todo

2 Criar 2 rankings considerando o tipoItem (S ou M) com filtro itemSustentavel


# Outros Achados

## Referente ao "GET /modulo-uasg/2_consultarOrgao

### CNPJs duplicados

In [ ]:
df = listar_orgaos()  # Obtém o DataFrame com os dados dos órgãos
coluna = 'cnpjCpfOrgao'  # Define a coluna a ser filtrada

# Filtra o DataFrame para incluir apenas linhas onde o CNPJ/CPF é diferente de '0'
df_filtrado = df[df[coluna] != '0']

# Identificando todas as duplicatas, incluindo a primeira ocorrência
duplicatas = df_filtrado.duplicated(subset=[coluna], keep=False)

# Filtrando o DataFrame para manter apenas as linhas duplicadas
df_filtrado_com_duplicatas = df_filtrado.loc[duplicatas]

# Exibindo as linhas duplicadas
display(df_filtrado_com_duplicatas)

### CNPJs Zerados

In [ ]:
df = listar_orgaos()  # Obtém o DataFrame com os dados dos órgãos
coluna = 'cnpjCpfOrgao'  # Define a coluna a ser filtrada
# Filtra o DataFrame para incluir apenas linhas onde o CNPJ/CPF é igual a '0'
df_cnpj_zerado = df[df[coluna] == '0']

# Exibindo as linhas duplicadas
display(df_cnpj_zerado)

## Valores Absurdos

In [ ]:
# prompt: ler o arquivo items_planejados_20240216.csv e transformar num dataframe
df_analise_valores = pd.read_csv('itens_planejados_20240216.csv')

In [ ]:
# prompt: Usando o DataFrame df_analise_valores: criar um boxplot com distribuição do valor total

import seaborn as sns
sns.boxplot(x=df_analise_valores["valorTotalItem"])


In [ ]:
# DF OUTLIERS
# Passo 1: Calcular Q1, Q3 e IQR
Q1 = df_analise_valores['valorTotalItem'].quantile(0.25)
Q3 = df_analise_valores['valorTotalItem'].quantile(0.75)
IQR = Q3 - Q1

fator_multiplicação = 1000

# Passo 2: Definir limites para outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + (1.5 * fator_multiplicação) * IQR

# Passo 3: Filtrar o DataFrame para incluir apenas outliers
df_outliers = df_analise_valores[df_analise_valores['valorTotalItem'] > limite_superior].copy()

# Ordenando df_outliers pela coluna 'valorTotalItem' em ordem decrescente
df_outliers = df_outliers.sort_values(by='valorTotalItem', ascending=False)

# Corrigindo a formatação de 'valorTotalItem'
df_outliers['valorTotalItem'] = df_outliers['valorTotalItem'].apply(lambda x: f"{x:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))


# Exibindo os outliers ordenados
display(df_outliers)
